In [1]:
import numpy as np
import pandas as pd

In [2]:
EPS = 1e-10
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true + EPS - y_pred) / y_true + EPS)) * 100    

In [3]:
def ds(y_true, y_pred):
    dts = 0

    for t in range(1, len(y_true)):
        if (y_true[t] - y_true[t-1]) * (y_pred[t] - y_pred[t-1]) > 0:
            dts += 1
    return 100 / len(y_true) * dts

In [4]:
def m(avg_absolute_errors):
    return np.sum(avg_absolute_errors) / len(avg_absolute_errors)

In [6]:
def r(avg_absolute_errors):
    _m = m(avg_absolute_errors)
    
    def s():
        squares = 0
       
        for i in range(len(avg_absolute_errors)):
            squares += np.square(avg_absolute_errors[i] - _m)
        
        return np.sqrt(squares / (len(avg_absolute_errors) - 1))
    
    
    return s() / _m

In [7]:
coins = [1442, 1445, 1456, 1446, 1453, 1447, 1452, 1443, 1457, 1451, 1460, 1448, 1454, 1477, 1449, 1444, 1450, 1474, 1455, 1465]

In [99]:
def make_submission(test_periods):
    mapes = {
        'coinId': [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: [],
    }
    dss = {
        'coinId': [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: [],
    }
    
    for coin in coins:
#         print(coin)
        y_true = pd.read_csv('../radpet/data/subm_int/'+str(coin)+'.csv', index_col='time')
        y_true = y_true[1:]
        y_pred = pd.read_csv('../radpet/data/subm_int/only_price_on_full/'+str(coin)+'.csv')
        y_pred.index = y_true.index
        y_pred['true_price'] = y_true['price']
        
        mapes['coinId'].append(coin)
        dss['coinId'].append(coin)
        
        i = 1
        for start, end in test_periods:
#             print(start,end)
            prices = y_pred.loc[(y_pred.index >= start) & (y_pred.index < end)]
            pred = prices.price
            pred.to_csv('subm/'+str(coin)+"_"+str(i)+".csv", index=False, header=False)
            
            mapes[i].append(mape(y_true=prices.true_price, y_pred=prices.price))
            dss[i].append(ds(y_true=prices.true_price, y_pred=prices.price))
            
            i += 1
            
    mapes = pd.DataFrame(mapes)
    mapes.to_csv('subm/metrics_MAPE.csv', index=False)
    
    
    dss = pd.DataFrame(dss)
    dss.to_csv('subm/metrics_DS.csv', index=False)

In [100]:
make_submission([['2018-02-01 00:00:00','2018-02-02 00:05:00'], ['2018-02-01 00:00:00','2018-02-02 00:05:00'], ['2018-02-01 00:00:00','2018-02-02 00:05:00'], ['2018-02-01 00:00:00','2018-02-02 00:05:00'], ['2018-02-01 00:00:00','2018-02-02 00:05:00']])